<a href="https://colab.research.google.com/github/PunchNFIT/prism-neuron/blob/main/PrismNeuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install transformers torch numpy

# Imports
import math, numpy as np
from transformers import pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
def prism_neuron(x, substrates):
    """
    x: int (0 for negative event, 1 for positive)
    substrates: list of {'w': float, 's': 0/1}
    returns: y_hat (float in [-1,1]), and debug tuple
    """
    # emotional valence
    v = 2 * x - 1
    z1 = v

    # reasoned pathway
    if substrates:
        z2 = sum(s['w']*s['s'] for s in substrates) / len(substrates)
    else:
        z2 = 0

    # activations
    g1 = math.tanh(z1)
    g2 = math.tanh(z2)

    # final (median = robust combination)
    y_hat = np.median([g1, g2])
    return y_hat, (v, z1, z2, g1, g2)

In [ ]:
# Example scenario: “He cheats”
x = 0
substrates = [
  {'w': 1.0, 's': 1},   # They have kids
  {'w': 0.8, 's': 1},   # Her parents’ divorce
  {'w': 0.6, 's': 1},   # Shared mortgage
]

y_hat, debug = prism_neuron(x, substrates)
print("Decision value ŷ =", y_hat)
print("Debug (v, z1, z2, g1, g2) =", debug)

Decision value ŷ = -0.04877869284395797
Debug (v, z1, z2, g1, g2) = (-1, -1, 0.7999999999999999, -0.7615941559557649, 0.6640367702678489)


In [ ]:
state = "stay" if y_hat > 0 else "leave"
conflict = abs(y_hat)  # how strong the pull/conflict is
print(f"State: {state}, Conflict level: {conflict:.2f}")

State: leave, Conflict level: 0.05


In [ ]:
# Initialize a lightweight generator
generator = pipeline("text-generation", model="distilgpt2")

# Build a prompt based *only* on your decision
prompt = (
  f"She knows he cheats but decides to {state}. "
  f"Conflict level: {conflict:.2f}. "
  "Write her inner dialogue in two sentences:"
)

# Generate
result = generator(prompt, max_length=60, num_return_sequences=1)
print(result[0]['generated_text'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


She knows he cheats but decides to leave. Conflict level: 0.05. Write her inner dialogue in two sentences: "I never went back to work."


"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I never went back to work."
"I


In [10]:
from transformers import pipeline, AutoTokenizer

# 1) Initialize the tokenizer and model
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Set the padding token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

generator = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=tokenizer,
    truncation=True,
    padding=True
)

# 2) Build your prompt once you have y_hat, state, conflict
prompt = (
    f"She knows he cheats but decides to {state}. "
    f"Conflict level: {conflict:.2f}. "
    "Write her inner dialogue in two varied sentences:"
)

# 3) Generate with sampling controls
response = generator(
    prompt,
    max_new_tokens=60,    # generate up to 60 tokens beyond the prompt
    temperature=0.8,      # higher temperature for more creativity (0.7–1.0)
    top_k=50,             # consider only the top 50 tokens at each step
    top_p=0.9,            # or nucleus sampling with p=0.9
    do_sample=True,       # turn on sampling (instead of greedy)
    num_return_sequences=1
)

print(response[0]["generated_text"])

Device set to use cpu


She knows he cheats but decides to leave. Conflict level: 0.05. Write her inner dialogue in two varied sentences: 1. I don't know who is fighting. 2. I don't know who is doing what he's doing. 3. I don't know who is going to go after me. 4. I don't know who is going to do what he's doing. 5. I don't know
